## Cell Number 1: *Imports*
Importing:
- basic libraries for Jupyter Notebooks
- `pathlib` for understanding where files are (e.g., `data/raw/dataset_test_csv.csv`)
- `pyarrow` for parsing; used as the engine for reading CSV files with `pandas`. 
    - This can make reading large CSV files faster and more memory-efficient compared to the default `pandas.read_csv()` engine.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from pandas import StringDtype, Int64Dtype, Float32Dtype
import pyarrow
import sys

# Import project configuration - add parent directory to path
sys.path.insert(0, '../')
from config import RAW_CSV, CLEANED_DATA_PARQUET, DATA_PROCESSED

# Make sure that all libraries loaded correctly from the `pip install`!
print("Libraries imported successfully!")
print(f"Using config.py for standardized project paths")


Libraries imported successfully!
Using config.py for standardized project paths


---

## Cell 1.5: *Quick Data Preview*
Let's preview the column names in our data and let's look at the shape of the data

In [2]:
print("=" * 80)
print("DATA LOADING CHECK")
print("=" * 80)

# Use config.py path for consistency
print(f"Loading from: {RAW_CSV}")
preview_df = pd.read_csv(
    RAW_CSV,
    engine='pyarrow',
)

# Calculate and display the total memory usage in bytes
preview_total_memory_bytes = preview_df.memory_usage(deep=True).sum()
# Convert to MB for easier reading
preview_total_memory_mb = preview_total_memory_bytes / ( 1024 * 1024 )
preview_mem_usage = f"Memory Usage: {preview_total_memory_mb:.2f} MB"

print(f"\nData loaded successfully!")

print(f"\nDATA SHAPE:")
print(f"Shape: {preview_df.shape}")

print(f"\nCOLUMN NAMES:")
print(f"Columns: {list(preview_df.columns)}")

print(f"\nFirst few rows:")
display(preview_df.head(5))
print(f"\n{preview_mem_usage}")

DATA LOADING CHECK
Loading from: /home/yler/GitHub/RaceLens/src/../data/raw/dataset_test_csv.csv

Data loaded successfully!

DATA SHAPE:
Shape: (60752, 30)

COLUMN NAMES:
Columns: ['registration_number', 'horse_name', 'track_id', 'race_date', 'Distance', 'race_number', 'race_type', 'course_type', 'country', 'Purse', 'Field_size', 'length_behind_at_poc_1', 'length_behind_at_poc_2', 'length_behind_at_poc_3', 'length_behind_at_poc_4', 'length_behind_at_poc_5', 'length_behind_at_finish', 'post_position', 'position_at_point_of_call_1', 'position_at_point_of_call_2', 'position_at_point_of_call_3', 'position_at_point_of_call_4', 'official_position', 'jockey_id', 'trainer_id', 'Earnings', 'equipment', 'Final Odds', 'favorite_indicator', 'speed_figure']

First few rows:


,registration_number,horse_name,track_id,race_date,Distance,race_number,race_type,course_type,country,Purse,...,position_at_point_of_call_3,position_at_point_of_call_4,official_position,jockey_id,trainer_id,Earnings,equipment,Final Odds,favorite_indicator,speed_figure
0,13008939,Restless Rambler,BKF,8/31/2025,4.32F,5,STK,D,USA,"$4,500.00",...,0,0,2,171618,246029,"$1,125.00",,3.0,N,60
1,13008939,Restless Rambler,WYO,8/9/2025,4.5F,6,CLM,D,USA,"$12,000.00",...,0,0,7,160633,153736,$0.00,F,10.8,N,65
2,13008939,Restless Rambler,WYO,7/12/2025,4.5F,8,CLM,D,USA,"$11,500.00",...,0,0,2,160633,153736,"$2,300.00",F,5.5,N,72
3,13008939,Restless Rambler,WYO,6/29/2025,4.5F,9,SOC,D,USA,"$10,500.00",...,0,0,7,18028,153736,$0.00,F,11.9,N,64
4,13010216,Libertarian,FAR,7/25/2025,7F,7,SST,D,USA,"$15,000.00",...,0,0,3,111515,39754,"$1,500.00",B,10.9,N,76



Memory Usage: 46.23 MB


---

## Cell Number 2: *Pandas Setup and DataType Config Setup*
This cell creates a consistent notebook format and defines the basic `df` configuration for loading data:

- **Display settings**: Limits row output, auto-summarizes wide frames, and sets a readable width for console output.
- **File path**: Points to the raw CSV file using `Path`.
- **Column selection**: `raw_usecols` lists only the columns needed for analysis, optimizing memory and load time.
- **Missing value tokens**: `na_tokens` defines all strings that should be treated as `NaN`, including common placeholders like `'NA'`, `'NULL'`, and `'—'`.
- **Data types**: `raw_dtypes` maps each column to its appropriate type (e.g., `StringDtype`, `Int64Dtype`, `Float32Dtype`) to ensure clean parsing and memory efficiency.

This setup makes the notebook readable and ensures the data is loaded efficiently.

In [3]:
# Display config for cell outputs
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 0)     # Auto-summarize wide frames
pd.set_option("display.width", 120)

# Use config.py path for consistency
raw_data_path = RAW_CSV
raw_usecols = [
    "registration_number","horse_name","track_id","race_date","Distance",
    "race_number","race_type","course_type","country","Purse","Field_size",
    "length_behind_at_poc_1","length_behind_at_poc_2","length_behind_at_poc_3",
    "length_behind_at_poc_4","length_behind_at_poc_5","length_behind_at_finish",
    "post_position","position_at_point_of_call_1","position_at_point_of_call_2",
    "position_at_point_of_call_3","position_at_point_of_call_4","official_position",
    "jockey_id","trainer_id","Earnings","equipment","Final Odds",
    "favorite_indicator","speed_figure"
]

na_tokens = ['NA','N/A','n/a','NULL','null','None','.','-','—','']

# Set the datatype for each column from the CSV file.
raw_dtypes = {
    "registration_number": StringDtype(),
    "horse_name": StringDtype(),
    "track_id": StringDtype(),
    "race_type": StringDtype(),
    "course_type": StringDtype(),
    "country": StringDtype(),
    "jockey_id": Int64Dtype(),
    "trainer_id": Int64Dtype(),
    "equipment": StringDtype(),
    "official_position": Int64Dtype(),
    "Final Odds": Float32Dtype(),
    "race_number": Int64Dtype(),
    "Field_size": Int64Dtype(),
    "favorite_indicator": StringDtype(),
    "speed_figure": Int64Dtype()
}

print("Configuration set successfully!")

Configuration set successfully!


---

## Cell Number 3: *Read*, *rename*, *clean*
This cell imports the horse racing data and preprocesses datatypes:
- **Header validation**: Ensures all expected columns (`raw_usecols`) are present in the CSV file.
- **Selective loading**: Reads only the necessary columns using `usecols`, applies custom `dtype`s, and parses `race_date`.
- **Column normalization**: Renames key columns to lowercase for consistency.
- **Distance cleanup**: Strips the `'F'` suffix from `distance` column and converts to numeric.
- **Currency cleanup**: Removes `$` and `,` from `purse` and `earnings` columns for math operations.
- **Whitespace handling**: Converts empty strings, or strings with only spaces, to `NaN` using regex.
- **Memory optimization**: Converts repeated text columns to `category` dtype when appropriate.

This sets up a clean, memory-efficient DataFrame experience for downstream analysis... as reading the CSV each time with 60k rows is memory-intense.

In [4]:
print("=" * 80)
print("CLEANED DATA OUTPUT")
print("=" * 80)

# Validate headers first in case something mismatches
file_cols = set(pd.read_csv(raw_data_path, nrows=0).columns)
missing = [c for c in raw_usecols if c not in file_cols]
if missing:
    raise ValueError(f"usecols missing in CSV: {missing}")

df = pd.read_csv(
    raw_data_path,
    usecols=raw_usecols,
    dtype=raw_dtypes,           # type: ignore[arg-type]
    parse_dates=["race_date"],
    na_values=na_tokens,
    keep_default_na=True,
    engine="pyarrow",         # Parser engine; optional; often faster
    encoding="utf-8"
)


# Normalize these column names we’ll use downstream
rename_map = {     # These columns are re-named (cleaned..) post-load, since they are not lower-cased.
    "Distance": "distance",
    "Purse": "purse",
    "Field_size": "field_size",
    "Earnings": "earnings",
    "Final Odds": "final_odds",
}
df.rename(columns=rename_map, inplace=True)

# Cleanse the `distance` column of the letter 'F' (e.g., "4.32F" -> 4.32)
if "distance" in df:
    dist_num = df["distance"].astype("string").str.extract(r"([+-]?\d+(?:\.\d+)?)", expand=False)
    df["distance"] = pd.to_numeric(dist_num, errors="coerce").astype(Float32Dtype())

# Cleanse the `purse` column of "$" and "," (e.g., $4,500.00 -> 4500.00) for clean math operations.
for money_col in ["purse", "earnings"]:
    if money_col in df:
        s = (df[money_col].astype("string")
             .str.replace(r"[\$,]", "", regex=True)
             .str.strip()
             .replace({"": np.nan}))
        df[money_col] = pd.to_numeric(s, errors="coerce").astype(Float32Dtype())

# Tidy strings
# Treat empty strings or strings with only spaces as `NaN`
df = df.replace(r'^\s*$', np.nan, regex=True)

# Categories for repeated text cols (a memory (efficiency) "win")
for cat_col in ["track_id", "race_type", "course_type", "equipment", "country"]:
    if cat_col in df and df[cat_col].nunique(dropna=True) <= 0.5 * len(df):
        df[cat_col] = df[cat_col].astype("category")


# Below is personal for me (optional); I like to see memory-output
# Calculate and display the total memory usage in bytes
total_memory_bytes = df.memory_usage(deep=True).sum()
# Convert to MB for easier reading
total_memory_mb = total_memory_bytes / ( 1024 * 1024 )
mem_usage = f"Memory Usage: {total_memory_mb:.2f} MB"


display(df.head(5))
print("=" * 80)
print(f"DATAFRAME SHAPE")
print("=" * 80)
print(f"\n{df.shape}\n")
print(f"\nShape = (Rows present, columns present)\n")

# Memory Usage and dtypes of every column
print("=" * 80)
print(f"COLUMN-BASED DATATYPES + MEMORY USAGE STAT")
print("=" * 80)
print(f"\n")
display(df.info(memory_usage="deep"))
print(f"\n")

# cleaned_df_for_export = df.copy()

CLEANED DATA OUTPUT


,registration_number,horse_name,track_id,race_date,distance,race_number,race_type,course_type,country,purse,field_size,length_behind_at_poc_1,length_behind_at_poc_2,length_behind_at_poc_3,length_behind_at_poc_4,length_behind_at_poc_5,length_behind_at_finish,post_position,position_at_point_of_call_1,position_at_point_of_call_2,position_at_point_of_call_3,position_at_point_of_call_4,official_position,jockey_id,trainer_id,earnings,equipment,final_odds,favorite_indicator,speed_figure
0,13008939,Restless Rambler,BKF,2025-08-31,4.32,5,STK,D,USA,4500.0,7,10,0,0,0,60,175,3,2,0,0,0,2,171618,246029,1125.0,<NA>,3.0,N,60
1,13008939,Restless Rambler,WYO,2025-08-09,4.5,6,CLM,D,USA,12000.0,8,0,0,0,0,70,860,6,1,0,0,0,7,160633,153736,0.0,F,10.8,N,65
2,13008939,Restless Rambler,WYO,2025-07-12,4.5,8,CLM,D,USA,11500.0,7,0,0,0,0,100,400,7,1,0,0,0,2,160633,153736,2300.0,F,5.5,N,72
3,13008939,Restless Rambler,WYO,2025-06-29,4.5,9,SOC,D,USA,10500.0,10,260,0,0,0,340,1225,7,9,0,0,0,7,18028,153736,0.0,F,11.9,N,64
4,13010216,Libertarian,FAR,2025-07-25,7.0,7,SST,D,USA,15000.0,6,150,100,0,0,50,160,4,3,2,0,0,3,111515,39754,1500.0,B,10.9,N,76


DATAFRAME SHAPE

(60752, 30)


Shape = (Rows present, columns present)

COLUMN-BASED DATATYPES + MEMORY USAGE STAT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60752 entries, 0 to 60751
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   registration_number          60752 non-null  string        
 1   horse_name                   60752 non-null  string        
 2   track_id                     60752 non-null  category      
 3   race_date                    60752 non-null  datetime64[ns]
 4   distance                     60752 non-null  Float32       
 5   race_number                  60752 non-null  Int64         
 6   race_type                    60752 non-null  category      
 7   course_type                  60752 non-null  category      
 8   country                      60752 non-null  category      
 9   purse                        60752 non-null  Float32  

None

##### ======== Quick Note about the `Tidy Strings` line in Cell 3 (on line 49) ========

In Cell 3, the line `df = df.replace(r'^\s*$', np.nan, regex=True)` is critical for treating ...  
        1. **empty strings**, or  
        2. **strings with only spaces**  

...as a **`NaN` value type**.


<br>

Since both *Excel* and *numpy* treat missing values as whitespace (empty strings), `numpy` does not accurately report the missing values in the `Equipment` and `favorite_indicator` columns. This results in an inaccurate print statement of **"No missing values"** — which affects the report. To solve this, I used a `regex` (Regular Expression) to treat whitespace(s) as a true `NaN` value-type.

*You can see the missing values below*:

---

## Cell Number 4: *Missing Values Check*
This cell scans for missing data across the cleaned `DataFrame`:
- **Summary table**: Builds a Missing Data (`missing_data`) `DataFrame` showing:
  - `Column`: name of each column
  - `Missing_Count`: total number of `NaN` entries
  - `Missing_Percentage`: percentage of missing values relative to total rows
- **Filter logic**: Displays only columns with at least one missing value, sorted by count.
- **Conditional output**:
  - If missing values exist, prints and displays them.
  - If none are found, print a confirmation message.
- **Memory usage report**: Uses my custom `mem_usage` string to report the `df` memory usage during cell execution, to show column types, and as a general memory footprint. This is mainly for my curosity.
  - See *Line 59* of *Cell 3* for the definition of `mem_usage`

This **check** ensures data quality before analysis begins.

In [5]:
print("=" * 80)
print("MISSING VALUES CHECK:")
print("=" * 80)

missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2).astype(str) + '%'
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False, ignore_index=True)

if len(missing_data) > 0:
    print("\nColumns with missing values:")
    display(missing_data)
else:
    print("\nNo missing values found!")

print(mem_usage)

MISSING VALUES CHECK:

Columns with missing values:


,Column,Missing_Count,Missing_Percentage
0,equipment,17621,29.0%
1,favorite_indicator,1,0.0%


Memory Usage: 19.90 MB


---

## Cell Number 5: *Duplicate Rows Check*
We want to check for duplicate horse rows ( using `df.duplicated()` ). 

`df.duplicated()` checks if an *entire row* is identical across all columns, in cases like:
``` csv
Row 1: Restless Rambler, BKF, 2025-08-31, 4.32F, 5, STK, D, ...
Row 2: Restless Rambler, BKF, 2025-08-31, 4.32F, 5, STK, D, ... (EXACT SAME)
```

In [6]:
print("=" * 80)
print("DUPLICATE ROWS ANALYSIS")
print("=" * 80)

duplicate_count = df.duplicated().sum()
print(f"\nTotal duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    print("\nExample of duplicate rows:")
    display(df[df.duplicated(keep=False)].head())

DUPLICATE ROWS ANALYSIS

Total duplicate rows: 0


---

---
---

---

## Cell Number 5.5: *Data Validation & Outlier Detection*

In [7]:
# Cell Number 5.5: *Data Validation & Outlier Detection*
print("=" * 80)
print("DATA VALIDATION & OUTLIER DETECTION")
print("=" * 80)

# Validate numeric ranges and collect outlier counts
numeric_cols = ['purse', 'earnings', 'field_size', 'distance', 'final_odds', 'speed_figure']
outlier_counts = {}  # Store outlier counts for each column

for col in numeric_cols:
    if col in df.columns:
        stats = df[col].describe()
        print(f"\n{col.upper()} STATS:")
        print(f"  Range: {stats['min']:.2f} - {stats['max']:.2f}")
        print(f"  Mean: {stats['mean']:.2f}, Std: {stats['std']:.2f}")
        
        # Check for outliers using IQR
        Q1 = stats['25%']
        Q3 = stats['75%']
        IQR = Q3 - Q1
        outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))][col].count()
        outlier_counts[col] = int(outliers)  # Store correctly for each column
        print(f"  Outliers: {outliers} ({outliers/len(df)*100:.2f}%)")

# Validate dates
print(f"\nDATE RANGE: {df['race_date'].min()} to {df['race_date'].max()}")
print(f"Future dates: {(df['race_date'] > pd.Timestamp.now()).sum()}")
print(f"  Note: Data appears to be from May-Sept 2025 (synthetic/test data)")

# Validate categorical values
cat_cols = ['track_id', 'race_type', 'course_type', 'country']
for col in cat_cols:
    if col in df.columns:
        unique_vals = df[col].nunique()
        print(f"\n{col}: {unique_vals} unique values")
        if unique_vals < 20:
            print(f"  Values: {df[col].value_counts().head(10).to_dict()}")

# Validate string columns
string_cols = ['horse_name', 'jockey_id', 'trainer_id']
for col in string_cols:
    if col in df.columns and df[col].dtype == 'object':
        print(f"\n{col} STRING VALIDATION:")
        # Check for unusual patterns
        avg_length = df[col].dropna().str.len().mean()
        max_length = df[col].dropna().str.len().max()
        min_length = df[col].dropna().str.len().min()
        print(f"  Length range: {min_length} - {max_length} (avg: {avg_length:.1f})")


# Validate position columns (excluding length_behind which is distance, not position)
position_cols = [col for col in df.columns 
                 if ('position' in col.lower() and 'length_behind' not in col.lower())]
print("\nPOSITION COLUMNS VALIDATION:")
for col in position_cols:
    if col in df.columns:
        valid_positions = df[col].between(0, df['field_size'], inclusive='both').sum()
        invalid = (~df[col].between(0, df['field_size'], inclusive='both') & df[col].notna()).sum()
        if invalid > 0:
            print(f"  {col}: {invalid} positions outside field size range")


print("\nFINAL DTYPE VALIDATION:")
expected_dtypes = {
    'distance': ('float32', 'Float32'),
    'purse': ('float32', 'Float32'),
    'earnings': ('float32', 'Float32'),
    'race_date': ('datetime64[ns]',),
    'track_id': ('category',),
    'race_type': ('category',)
}

for col, expected_types in expected_dtypes.items():
    if col in df.columns:
        actual = str(df[col].dtype)
        # Accept both numpy float32 and pandas Float32 (nullable)
        if any(exp in actual for exp in expected_types):
            print(f"   {col}: {actual}")
        else:
            print(f"  ✗ {col}: Expected {expected_types}, got {actual}")


print("\nCROSS-COLUMN VALIDATION:")

# Winners should have earnings > 0
winners_no_earnings = ((df['official_position'] == 1) & (df['earnings'] == 0)).sum()
if winners_no_earnings > 0:
    print(f"  Warning: {winners_no_earnings} winners with zero earnings")

# Favorites should have lower odds
fav_high_odds = ((df['favorite_indicator'] == 'Y') & (df['final_odds'] > 5)).sum()
if fav_high_odds > 0:
    print(f"  Warning: {fav_high_odds} favorites with odds > 5")

# Field size should match position values
max_pos = df[['position_at_point_of_call_1', 'position_at_point_of_call_2', 
              'position_at_point_of_call_3', 'position_at_point_of_call_4']].max(axis=1)
mismatched = (max_pos > df['field_size']).sum()
if mismatched > 0:
    print(f"  Warning: {mismatched} races with positions > field size")


# Save the validation results to a JSON file
validation_summary = {
    'total_rows': len(df),
    'total_columns': len(df.columns),
    'missing_values': df.isnull().sum().to_dict(),
    'outlier_counts': outlier_counts,  # Now correctly stores each column's count
    'memory_usage_mb': total_memory_mb,
    'date_range': f"{df['race_date'].min()} to {df['race_date'].max()}",
    'data_note': 'Synthetic/test data from May-Sept 2025'
}

# Save as JSON for reference
import json
# Use config.py paths
output_dir = DATA_PROCESSED
output_dir.mkdir(parents=True, exist_ok=True)
validation_path = output_dir / 'data_validation_summary.json'
with open(validation_path, 'w') as f:
    json.dump(validation_summary, f, indent=2, default=str)
print(f"\nValidation summary saved to {validation_path}")

DATA VALIDATION & OUTLIER DETECTION

PURSE STATS:
  Range: 2250.00 - 5000000.00
  Mean: 39751.72, Std: 114524.93
  Outliers: 6648 (10.94%)

EARNINGS STATS:
  Range: 0.00 - 3100000.00
  Mean: 5345.63, Std: 23384.68
  Outliers: 6741 (11.10%)

FIELD_SIZE STATS:
  Range: 1.00 - 19.00
  Mean: 7.43, Std: 1.88
  Outliers: 94 (0.15%)

DISTANCE STATS:
  Range: 2.00 - 25.00
  Mean: 6.80, Std: 1.42
  Outliers: 351 (0.58%)

FINAL_ODDS STATS:
  Range: 0.00 - 273.40
  Mean: 13.34, Std: 17.83
  Outliers: 5808 (9.56%)

SPEED_FIGURE STATS:
  Range: 0.00 - 999.00
  Mean: 69.59, Std: 69.57
  Outliers: 1472 (2.42%)

DATE RANGE: 2025-05-01 00:00:00 to 2025-09-30 00:00:00
Future dates: 0
  Note: Data appears to be from May-Sept 2025 (synthetic/test data)

track_id: 81 unique values

race_type: 29 unique values

course_type: 9 unique values
  Values: {'D': 42587, 'T': 10440, 'E': 5157, 'I': 1617, 'O': 814, 'M': 87, 'C': 44, 'S': 5, 'B': 1}

country: 5 unique values
  Values: {'USA': 53221, 'CAN': 7464, 'AUS'

---

In [8]:
# Add this as a new cell after Cell 5.5

# Cell 5.6: DNF (Did Not Finish) Analysis - CRITICAL
print("=" * 80)
print("CRITICAL DATA CHECK: DNF (Did Not Finish) Analysis")
print("=" * 80)

# Check for 999 speed values (DNF indicator)
dnf_races = (df['speed_figure'] == 999).sum()
total_races = len(df)
dnf_percentage = (dnf_races / total_races) * 100

print(f"\nDNF Statistics:")
print(f"Total races: {total_races:,}")
print(f"DNF races (speed=999): {dnf_races:,}")
print(f"DNF percentage: {dnf_percentage:.2f}%")

# Show distribution of non-DNF speeds
valid_speeds = df[df['speed_figure'] != 999]['speed_figure']
print(f"\nValid Speed Figure Statistics (excluding DNFs):")
print(f"Count: {len(valid_speeds):,}")
print(f"Mean: {valid_speeds.mean():.2f}")
print(f"Std: {valid_speeds.std():.2f}")
print(f"Min: {valid_speeds.min()}")
print(f"Max: {valid_speeds.max()}")

# Add a new column to flag DNF races
df['is_dnf'] = df['speed_figure'] == 999

# Show examples of DNF races
print(f"\nSample DNF races:")
dnf_sample = df[df['is_dnf']][['horse_name', 'race_date', 'official_position', 'speed_figure', 'field_size']].head(10)
display(dnf_sample)

print("\nWARNING: Speed figure of 999 indicates DNF (Did Not Finish)")
print("These MUST be handled separately in analysis to avoid skewing averages!")

CRITICAL DATA CHECK: DNF (Did Not Finish) Analysis

DNF Statistics:
Total races: 60,752
DNF races (speed=999): 311
DNF percentage: 0.51%

Valid Speed Figure Statistics (excluding DNFs):
Count: 60,441
Mean: 64.80
Std: 19.94
Min: 0
Max: 121

Sample DNF races:


,horse_name,race_date,official_position,speed_figure,field_size
16,Inagoodway,2025-07-12,6,999,6
407,Salvator Mundi,2025-08-01,6,999,6
1032,Too Crowded,2025-07-22,8,999,9
1305,Rain,2025-07-13,5,999,5
2289,Evie's Prince,2025-07-16,4,999,9
2290,Evie's Prince,2025-06-04,2,999,7
2291,Evie's Prince,2025-05-10,7,999,8
2436,Trust Fund Baby,2025-08-27,10,999,10
2928,Pomeroy Haze,2025-08-29,7,999,7
2952,Awesome Annmarie,2025-09-23,7,999,7



These MUST be handled separately in analysis to avoid skewing averages!


---

# Data Cleaning Notebook Summary
This notebook served as a detailed data-cleaning-and-preparation outline for the raw horse racing dataset. Here’s a summary of what was done to the original `dataset_test` file:
 
## 1. Library Imports & Setup
- Imported essential libraries: `pandas`, `numpy`, `pathlib`, and `pyarrow` for efficient data handling.
- Set pandas display options for better readability in the notebook.
 
## 2. Data Loading Configuration
- Defined the path to the raw CSV file and selected only the necessary columns for analysis (`raw_usecols`).
- Specified a comprehensive list of tokens to be treated as missing values (`na_tokens`).
- Created a `raw_dtypes` dictionary to explicitly set data types for each column, optimizing memory.
 
## 3. Data Preview & Validation
- Previewed the raw data to check column names, shape, and memory usage.
- Confirmed that all expected columns are present in the CSV before loading.
 
## 4. Data Cleaning & Transformation
- Loaded the data with pre-defined `dtype` types and `usecols` for efficiency.
- Renamed key columns to snake_case for consistency (e.g., `Distance` -> `distance`).
- Cleaned the `distance` column by extracting numeric values and converting to float.
- Cleaned currency columns (`purse`, `earnings`) by removing `$` and `,`, then converting to float type.
- Used a `regex` to convert empty strings and whitespace-only cells to `NaN` type across the DataFrame, ensuring accurate missing-value reporting.
- Tidied string columns by stripping whitespace and standardizing case (e.g., `favorite_indicator`).
- Converted repeated text columns to the "`category`" `dtype` for memory efficiency.
 
## 5. Data Quality Checks
- Checked for missing values in all columns and reported their counts and percentages.
- Checked for duplicate rows if any were present -- there was not.
 
---
### Outcome
- The resulting DataFrame is clean, memory-efficient, and ready for downstream analysis and modeling.
- All major sources of dirty data (missing values, inconsistent types, extra whitespace, currency formatting, and duplicates) have been addressed.
 
*-This notebook provides a detailed summary of how I typically approach cleaning any dataset. I chose a Jupyter Notebook over a standalone Python file for writing the logic to clean the horse racing data because the cell-based presentation approach used in Jupyter is excellent for both technical and non-technical audiences.*

*-To save time and memory efficiency from repeating this cleaning process in the `notebooks/` section, I am going to export the cleaned dataframe made in this Notebook as a new CSV file into the `data/processed/` directory. In using this Jupyter Notebook, I am able to clearly outline my logic/reasoning for treating the data the way I did and how I explored the data (which will be pursued more in-depth in the `notebooks/` section)*

*-Lastly, to show the alternative of using Jupyter envs to clean data, I have also included an object-oriented / class-based approach in pure Python (see `src/cleaning.py`). This OOP (Object-oriented Programming) approach is the winner in reusability, as writing the cleaning scripting logic as a class or function allows for modular re-use of each component. However, enough of my OOP lecture. Let's move on.*

*-Below, I will now export a copy of the cleaned dataframe*

### ================ Export cleaned Data ==================

In [9]:
# In data_cleaning.ipynb, before saving to Parquet, ensure:
df['is_dnf'] = df['speed_figure'] == 999

display(df.head(n=10)) # Verify that the data is still "cleaned" before export
display(df.info(memory_usage="deep"))
print(df.dtypes)

,registration_number,horse_name,track_id,race_date,distance,race_number,race_type,course_type,country,purse,field_size,length_behind_at_poc_1,length_behind_at_poc_2,length_behind_at_poc_3,length_behind_at_poc_4,length_behind_at_poc_5,length_behind_at_finish,post_position,position_at_point_of_call_1,position_at_point_of_call_2,position_at_point_of_call_3,position_at_point_of_call_4,official_position,jockey_id,trainer_id,earnings,equipment,final_odds,favorite_indicator,speed_figure,is_dnf
0,13008939,Restless Rambler,BKF,2025-08-31,4.32,5,STK,D,USA,4500.0,7,10,0,0,0,60,175,3,2,0,0,0,2,171618,246029,1125.0,<NA>,3.0,N,60,False
1,13008939,Restless Rambler,WYO,2025-08-09,4.5,6,CLM,D,USA,12000.0,8,0,0,0,0,70,860,6,1,0,0,0,7,160633,153736,0.0,F,10.8,N,65,False
2,13008939,Restless Rambler,WYO,2025-07-12,4.5,8,CLM,D,USA,11500.0,7,0,0,0,0,100,400,7,1,0,0,0,2,160633,153736,2300.0,F,5.5,N,72,False
3,13008939,Restless Rambler,WYO,2025-06-29,4.5,9,SOC,D,USA,10500.0,10,260,0,0,0,340,1225,7,9,0,0,0,7,18028,153736,0.0,F,11.9,N,64,False
4,13010216,Libertarian,FAR,2025-07-25,7.0,7,SST,D,USA,15000.0,6,150,100,0,0,50,160,4,3,2,0,0,3,111515,39754,1500.0,B,10.9,N,76,False
5,13010216,Libertarian,FAR,2025-07-13,6.0,5,STR,D,USA,8000.0,9,280,0,0,0,450,600,1,8,1,0,0,4,158119,39754,640.0,B,8.3,N,66,False
6,13010216,Libertarian,HCN,2025-06-27,6.0,6,CLM,D,USA,5500.0,8,720,470,0,0,60,0,4,8,6,0,0,1,158119,39754,3300.0,B,4.0,N,69,False
7,13010216,Libertarian,HCN,2025-06-13,5.0,1,CLM,D,USA,5500.0,5,450,600,0,0,600,355,1,4,3,0,0,3,158119,39754,660.0,B,2.5,N,72,False
8,13019678,Blue Jeans N Beer,LBG,2025-09-07,6.0,3,SOC,D,CAN,5494.0,4,0,50,0,0,20,575,2,1,2,0,0,4,158434,966218,330.0,F,2.3,N,63,False
9,13019678,Blue Jeans N Beer,GPR,2025-07-26,6.0,2,STR,D,CAN,5255.0,5,0,20,0,0,1200,2425,1,1,3,0,0,4,16075,966218,315.0,<NA>,2.8,N,23,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60752 entries, 0 to 60751
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   registration_number          60752 non-null  string        
 1   horse_name                   60752 non-null  string        
 2   track_id                     60752 non-null  category      
 3   race_date                    60752 non-null  datetime64[ns]
 4   distance                     60752 non-null  Float32       
 5   race_number                  60752 non-null  Int64         
 6   race_type                    60752 non-null  category      
 7   course_type                  60752 non-null  category      
 8   country                      60752 non-null  category      
 9   purse                        60752 non-null  Float32       
 10  field_size                   60752 non-null  Int64         
 11  length_behind_at_poc_1       60752 non-nu

None

registration_number    string[python]
horse_name             string[python]
track_id                     category
race_date              datetime64[ns]
distance                      Float32
                            ...      
equipment                    category
final_odds                    Float32
favorite_indicator     string[python]
speed_figure                    Int64
is_dnf                        boolean
Length: 31, dtype: object


##### Export

In [10]:
# Set this to True to overwrite existing files
FORCE_OVERWRITE = True

saved_files = []

# Use config.py paths for consistency
output_dir = DATA_PROCESSED
output_path = CLEANED_DATA_PARQUET
output_dir.mkdir(parents=True, exist_ok=True)

if output_path.exists() and not FORCE_OVERWRITE:
    print(f"File '{output_path.name}' already exists in {output_dir}.")
    print("Options:")
    print("  1. Set FORCE_OVERWRITE = True and re-run this cell to overwrite.")
    print("  2. Delete the file manually and re-run this cell.")
    print("  3. Skip this step and continue with analysis.")
    print("No changes made.")
else:
    if output_path.exists() and FORCE_OVERWRITE:
        print(f"Overwriting existing file: {output_path.name}")
    
    # Export to Parquet format - preserves all dtypes including datetime, categories, etc.
    df.to_parquet(output_path, index=False, engine='pyarrow')
    saved_files.append(output_path.name)
    print(f"{output_path.name} saved to {str(output_dir)}!")
    print(f"All dtypes preserved in Parquet format")

Overwriting existing file: cleaned_data.parquet
cleaned_data.parquet saved to /home/yler/GitHub/RaceLens/src/../data/processed!
All dtypes preserved in Parquet format
cleaned_data.parquet saved to /home/yler/GitHub/RaceLens/src/../data/processed!
All dtypes preserved in Parquet format


##### Verify export

In [11]:
# Check to make sure the file saved
existing_files = list(output_dir.glob("*.parquet"))

if existing_files and len(saved_files) > 0:
    print("Checking for files by name...")
    for file in existing_files:
        # Check if any Parquet files in `output_dir` match `saved_files[output_path.name]`
        if file.name in saved_files:
            print(f"\n File '{file.name}' exists in {output_dir}.")
            print("\n File was saved correctly.")
            file_size = file.stat().st_size / (1024 * 1024)
            print(f"\n File's size = {file_size:.2f} MB")
            
            # Quick verification that dtypes are preserved
            test_df = pd.read_parquet(file)
            print(f"\n Verification: DataFrame loaded with {len(test_df)} rows and {len(test_df.columns)} columns")
            print(f" Sample dtypes preserved: race_date={test_df['race_date'].dtype}, track_id={test_df['track_id'].dtype}")
        else:
            print(f"\n⚠ File by name of '{file.name}' not found in {output_dir}")
else:
    print(f"'Saved_files' OR 'existing_files' is empty...\nHave you executed the cell above to save the file?")

Checking for files by name...

 File 'cleaned_data.parquet' exists in /home/yler/GitHub/RaceLens/src/../data/processed.

 File was saved correctly.

 File's size = 1.71 MB

 Verification: DataFrame loaded with 60752 rows and 31 columns
 Sample dtypes preserved: race_date=datetime64[ns], track_id=category

 Verification: DataFrame loaded with 60752 rows and 31 columns
 Sample dtypes preserved: race_date=datetime64[ns], track_id=category


---

#### Proceed to Notebook 01 (`01_data_exploration.ipynb`)in `notebooks/` dir.